In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Use GPU
torch.cuda.is_available()
device = torch.device("cuda:0")
print(device)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

# Data preprocessing. Set to True if need to change something in training data.
REBUILD_DATA = True

class DogsVSCats(): 
   # ////////////////////////////////////////////////////////////////////////////////////////////////////////

    IMG_SIZE = 512 # Image will be 50x50 
    CATS = "D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//ax1_3//" # Path to cats images
    DOGS = "D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//sag1_3//" # Path to dogs images
    
   # ////////////////////////////////////////////////////////////////////////////////////////////////////////
    LABELS = {CATS: 0, DOGS: 1} # Represent cats as zeros, dogs as ones
    training_data = [] # Will be filled with preprocessed images of cats and dogs and their labels respectively
    
    # To see if data well balanced create counters for classes of images
    catcount = 0 # Counter for cats as we append training_data
    dogcount = 0 # same for dogs
    
    def make_training_data(self):
        for label in self.LABELS: # Iteratare through the keys of dictionarie LABELS
            print(label)
            # Iterate throught all images in directories CATS and DOGS
            for f in tqdm(os.listdir(label)): # Access to the directories
                if "jpg" in f: # Ignore all file that are not images
                    try:
                        path = os.path.join(label, f) # Join path to image and image's name in a single string
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) # Read images from path and convert to the grey scale. MAKES AN ARRAY!
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE)) # Resize images to 50x50
                        # Add all data in one-hot vectors to the list
                        # np.eye(2) - one-hot for 2 classes, array 2x2
                        # np.eye(10)[7] returns an array with 10 elements and element with index 7 is one, rest are zeroes
                        # np.eye(2)[self.LABELS[label]] - convert all cats and dogs to one-hot
                        self.training_data.append([np.array(img),np.eye(2)[self.LABELS[label]]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1
                    except Exception as e:
                        print(str(e))
                        #pass
                
        np.random.shuffle(self.training_data)
        np.save("D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//training_data.npy", self.training_data)
        print("Cats: ", self.catcount)
        print("Dogs: ", self.dogcount)

if REBUILD_DATA:
    dogvcats = DogsVSCats()
    dogvcats.make_training_data()
    
training_data = np.load("D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//training_data.npy", allow_pickle=True)
print("Number of samples: ", len(training_data))

print("DATA PREPROCESSED")

# Define neural network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim=1)
    
net = Net().to(device)

print("Network Architecture")
print(net)

# Define optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X / 255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X) * VAL_PCT)

print("Validation size: ", val_size)

# Define number of training and test samples
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print("Number of training samples: ", len(train_X))
print("Number of test samples: ", len(test_X))

# Training
BATCH_SIZE = 100
EPOCHS = 10

def train(net):
    print("begin the training")
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            #print(i, i + BATCH_SIZE)
            batch_X = train_X[i: i + BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i: i + BATCH_SIZE]
            
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()
            
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        
        print("Epoch: ", epoch, "Loss: ", int(loss))
    

    print("Loss value: ", int(loss))

# Accuracy test
def test(net):
    print("begin the test")
    correct = 0
    total = 0

    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]

            #print("Real class: ", real_class, "Net out: ", net_out)

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct / total, 3))

# Use GPU
print("Use GPU")

train(net)
test(net)

print("DONE")

cuda:0
Running on CPU
D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//ax1_3//


100%|██████████| 78/78 [00:00<00:00, 157.26it/s]


D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//sag1_3//


100%|██████████| 67/67 [00:00<00:00, 163.41it/s]


Cats:  78
Dogs:  67
Number of samples:  145
DATA PREPROCESSED
torch.Size([128, 2, 2])
Network Architecture
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


RuntimeError: shape '[-1, 50, 50]' is invalid for input of size 38010880

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

print("Imported")

Imported


In [2]:
# Use GPU
torch.cuda.is_available()
device = torch.device("cuda:0")
print(device)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

cuda:0
Running on CPU


In [3]:
# Data preprocessing. Set to True if need to change something in training data.
REBUILD_DATA = True

class DogsVSCats(): 
    IMG_SIZE = 50 # Image will be 50x50
    CATS = "D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//Cat//" # Path to cats images
    DOGS = "D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//Dog//" # Path to dogs images
    LABELS = {CATS: 0, DOGS: 1} # Represent cats as zeros, dogs as ones
    training_data = [] # Will be filled with preprocessed images of cats and dogs and their labels respectively
    
    # To see if data well balanced create counters for classes of images
    catcount = 0 # Counter for cats as we append training_data
    dogcount = 0 # same for dogs
    
    def make_training_data(self):
        for label in self.LABELS: # Iteratare through the keys of dictionarie LABELS
            print(label)
            # Iterate throught all images in directories CATS and DOGS
            for f in tqdm(os.listdir(label)): # Access to the directories
                if "jpg" in f: # Ignore all file that are not images
                    try:
                        path = os.path.join(label, f) # Join path to image and image's name in a single string
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) # Read images from path and convert to the grey scale. MAKES AN ARRAY!
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE)) # Resize images to 50x50
                        # Add all data in one-hot vectors to the list
                        # np.eye(2) - one-hot for 2 classes, array 2x2
                        # np.eye(10)[7] returns an array with 10 elements and element with index 7 is one, rest are zeroes
                        # np.eye(2)[self.LABELS[label]] - convert all cats and dogs to one-hot
                        self.training_data.append([np.array(img),np.eye(2)[self.LABELS[label]]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1
                    except Exception as e:
                        print("Exeption: ", str(e))
                        #pass
                
        np.random.shuffle(self.training_data)
        np.save("D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//training_data.npy", self.training_data)
        print("Cats: ", self.catcount)
        print("Dogs: ", self.dogcount)

if REBUILD_DATA:
    dogvcats = DogsVSCats()
    dogvcats.make_training_data()
    
training_data = np.load("D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//training_data.npy", allow_pickle=True)
print("Number of samples: ", len(training_data))

print("DATA PREPROCESSED")

D://Projects//test//data//catsvsdogs//kagglecatsanddogs_3367a//PetImages//Cat//


  1%|▎                                    | 101/12500 [00:00<00:24, 508.85it/s]

Exeption:  OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize



  4%|█▎                                   | 447/12500 [00:00<00:24, 501.29it/s]

Exeption:  OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize



...

Exeption:  OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize



 93%|████████████████████████████████▍  | 11567/12500 [00:21<00:01, 515.04it/s]

Exeption:  OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize



100%|███████████████████████████████████| 12500/12500 [00:23<00:00, 537.20it/s]


Cats:  12476
Dogs:  12470
Number of samples:  24946
DATA PREPROCESSED


[array([[102,  91,  73, ..., 126, 106, 123],
       [ 86,  95, 121, ..., 101, 109, 141],
       [ 98, 120, 115, ..., 132, 139, 139],
       ...,
       [155, 247, 238, ..., 238, 255, 224],
       [240, 184, 250, ..., 200, 252, 253],
       [250, 255, 178, ..., 255, 255, 221]], dtype=uint8)
 array([0., 1.])]


In [5]:
# Define neural network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim=1)
    
net = Net().to(device)

print("Network Architecture")
print(net)

torch.Size([128, 2, 2])
Network Architecture
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [6]:
# Define optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X / 255.0
y = torch.Tensor([i[1] for i in training_data])

print("Transition complete")

Transition complete


In [7]:
VAL_PCT = 0.1
val_size = int(len(X) * VAL_PCT)

print("Validation size: ", val_size)

# Define number of training and test samples
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print("Number of training samples: ", len(train_X))
print("Number of test samples: ", len(test_X))

Validation size:  2494
Number of training samples:  22452
Number of test samples:  2494


In [9]:
# Training
BATCH_SIZE = 100
EPOCHS = 3

def train(net):
    print("begin the training")
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            #print(i, i + BATCH_SIZE)
            batch_X = train_X[i: i + BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i: i + BATCH_SIZE]
            
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()
            
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        
        print("Epoch: ", epoch, "Loss: ", float(loss))
    

    print("Loss value: ", float(loss))

train(net)

print("Training complete")

begin the training



  0%|                                                  | 0/225 [00:00<?, ?it/s]

torch.Size([128, 2, 2])



  0%|▏                                         | 1/225 [00:01<07:02,  1.88s/it]

torch.Size([128, 2, 2])


...

torch.Size([128, 2, 2])



100%|███████████████████████████████████████▊| 224/225 [05:58<00:01,  1.75s/it]

torch.Size([128, 2, 2])



100%|████████████████████████████████████████| 225/225 [05:59<00:00,  1.55s/it]

Epoch:  2 Loss:  0
Loss value:  0
Training complete


In [10]:
# Accuracy test
def test(net):
    print("begin the test")
    correct = 0
    total = 0

    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]

            #print("Real class: ", real_class, "Net out: ", net_out)

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct / total, 3))


test(net)

print("DONE")

begin the test



  0%|                                                 | 0/2494 [00:00<?, ?it/s]

torch.Size([128, 2, 2])



  0%|                                         | 1/2494 [00:00<06:23,  6.49it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])


...

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])



100%|█████████████████████████████████████▉| 2493/2494 [01:30<00:00, 16.83it/s]

torch.Size([128, 2, 2])



100%|██████████████████████████████████████| 2494/2494 [01:30<00:00, 27.55it/s]

Accuracy:  0.738
DONE
